*Please use google colaboratory to run this notebook

# Create one single CSV
https://www.kaggle.com/fmitchell259/create-corona-csv-file

In [0]:
import numpy as np
import pandas as pd
import os
import json
import glob
import sys
sys.path.insert(0, "../")

root_path = 'C:\\Users\\angechowdhury\\OneDrive - Deloitte (O365D)\\Documents\\covid research'



In [0]:
#  Get all the files saved into a list and then iterate over them like below to extract relevant information
# hold this information in a dataframe and then move forward from there. 


In [0]:
# Just set up a quick blank dataframe to hold all these medical papers. 

corona_features = {"doc_id": [None], "source": [None], "title": [None],
                  "abstract": [None], "text_body": [None]}
corona_df = pd.DataFrame.from_dict(corona_features)


In [0]:
corona_df


,doc_id,source,title,abstract,text_body
0,None,None,None,None,None


In [0]:
 #Cool so dataframe now set up, lets grab all the json file names. 

# For this we can use the very handy glob library

json_filenames = glob.glob(f'{root_path}/**/*.json', recursive=True)

In [0]:
len(json_filenames)

14097

In [0]:
def return_corona_df(json_filenames, df, source):

    for file_name in json_filenames:

        row = {"doc_id": None, "source": None, "title": None,
              "abstract": None, "text_body": None}

        with open(file_name) as json_data:
            data = json.load(json_data)

            row['doc_id'] = data['paper_id']
            row['title'] = data['metadata']['title']

            # Now need all of abstract. Put it all in 
            # a list then use str.join() to split it
            # into paragraphs. 

            abstract_list = [data['abstract'][x]['text'] for x in range(len(data['abstract']) - 1)]
            abstract = "\n ".join(abstract_list)

            row['abstract'] = abstract

            # And lastly the body of the text. For some reason I am getting an index error
            # In one of the Json files, so rather than have it wrapped in a lovely list
            # comprehension I've had to use a for loop like a neanderthal. 
            
            # Needless to say this bug will be revisited and conquered. 
            
            body_list = []
            for _ in range(len(data['body_text'])):
                try:
                    body_list.append(data['body_text'][_]['text'])
                except:
                    pass

            body = "\n ".join(body_list)
            
            row['text_body'] = body
            
            # Now just add to the dataframe. 
            
            if source == 'b':
                row['source'] = "biorxiv_medrxiv"
            elif source == "c":
                row['source'] = "common_use_sub"
            elif source == "n":
                row['source'] = "non_common_use"
            elif source == "p":
                row['source'] = "pmc_custom_license"
            
            df = df.append(row, ignore_index=True)
    
    return df

In [0]:
corona_df = return_corona_df(json_filenames, corona_df, 'b')


In [0]:
corona_df.shape

(14098, 5)

In [0]:
corona_out = corona_df.to_csv('kaggle_covid-19_open_csv_format.csv')

# Create Seperate CSV
https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv

In [0]:
import os
import json
from pprint import pprint
from copy import deepcopy

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [0]:
def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(name)
        else:
            name_ls.append(name)
    
    return ", ".join(name_ls)

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors', 'venue', 'year']]
        formatted.append(", ".join(formatted_ls))

    return "; ".join(formatted)

In [0]:
 def load_files(dirname):
    filenames = os.listdir(dirname)
    raw_files = []

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        raw_files.append(file)
    
    return raw_files

def generate_clean_df(all_files):
    cleaned_files = []
    
    for file in tqdm(all_files):
        features = [
            file['paper_id'],
            file['metadata']['title'],
            format_authors(file['metadata']['authors']),
            format_authors(file['metadata']['authors'], 
                           with_affiliation=True),
            format_body(file['abstract']),
            format_body(file['body_text']),
            format_bib(file['bib_entries']),
            file['metadata']['authors'],
            file['bib_entries']
        ]

        cleaned_files.append(features)

    col_names = ['paper_id', 'title', 'authors',
                 'affiliations', 'abstract', 'text', 
                 'bibliography','raw_authors','raw_bibliography']

    clean_df = pd.DataFrame(cleaned_files, columns=col_names)
    clean_df.head()
    
    return clean_df

In [0]:
biorxiv_dir = 'C:\\Users\\angechowdhury\\OneDrive - Deloitte (O365D)\\Documents\\covid research\\biorxiv_medrxiv\\'
filenames = os.listdir(biorxiv_dir)
print("Number of articles retrieved from biorxiv:", len(filenames))

Number of articles retrieved from biorxiv: 1053


In [0]:
all_files = []

for filename in filenames:
    filename = biorxiv_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files.append(file)


In [0]:
file = all_files[0]
print("Dictionary keys:", file.keys())

Dictionary keys: dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])


In [0]:
pprint(file['abstract'])

[{'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'word count: 194 22 Text word count: 5168 23 24 25 author/funder. '
          'All rights reserved. No reuse allowed without permission. Abstract '
          '27 The positive stranded RNA genomes of picornaviruses comprise a '
          'single large open reading 28 frame flanked by 5′ and 3′ '
          'untranslated regions (UTRs). Foot-and-mouth disease virus (FMDV) 29 '
          'has an unusually large 5′ UTR (1.3 kb) containing five structural '
          'domains. These include the 30 internal ribosome entry site (IRES), '
          'which facilitates initiation of translation, and the cis-acting 31 '
          'replication element (cre). Less well characterised structures are a '
          '5′ terminal 360 nucleotide 32 stem-loop, a variable length '
          'poly-C-tract of approximately 100-200 nucleotides and a series of '
          '33 two to four tandemly repeated pseudoknots (PKs). We investigated

In [0]:
print("body_text type:", type(file['body_text']))
print("body_text length:", len(file['body_text']))
print("body_text keys:", file['body_text'][0].keys())

body_text type: <class 'list'>
body_text length: 20
body_text keys: dict_keys(['text', 'cite_spans', 'ref_spans', 'section'])


In [0]:
print("body_text content:")
pprint(file['body_text'][:2], depth=3)

body_text content:
[{'cite_spans': [],
  'ref_spans': [{...}],
  'section': '',
  'text': 'VP3, and VP0 (which is further processed to VP2 and VP4 during '
          'virus assembly) (6). The P2 64 and P3 regions encode the '
          'non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro '
          'and 4 structural protein-coding region is replaced by reporter '
          'genes, allow the study of genome 68 replication without the '
          'requirement for high containment (9, 10) ( figure 1A ).'},
 {'cite_spans': [{...}, {...}, {...}, {...}, {...}, {...}],
  'ref_spans': [],
  'section': '70',
  'text': 'The FMDV 5′ UTR is the largest known picornavirus UTR, comprising '
          'approximately 1300 71 nucleotides and containing several highly '
          'structured regions. The first 360 nucleotides at the 5′ 72 end are '
          'predicted to fold into a single large stem loop termed the '
          'S-fragment, followed by a The PKs were originally predicted 

In [0]:
texts = [(di['section'], di['text']) for di in file['body_text']]
texts_di = {di['section']: "" for di in file['body_text']}
for section, text in texts:
    texts_di[section] += text

pprint(list(texts_di.keys()))

['',
 '70',
 '120',
 '135',
 '136',
 '144',
 '301',
 'Function of the PKs in replication is dependent on downstream interactions '
 'and 350',
 '368',
 '468',
 '479']


In [0]:
body = ""

for section, text in texts_di.items():
    body += section
    body += "\n\n"
    body += text
    body += "\n\n"

print(body[:3000])




VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structural protein-coding region is replaced by reporter genes, allow the study of genome 68 replication without the requirement for high containment (9, 10) ( figure 1A ).

70

The FMDV 5′ UTR is the largest known picornavirus UTR, comprising approximately 1300 71 nucleotides and containing several highly structured regions. The first 360 nucleotides at the 5′ 72 end are predicted to fold into a single large stem loop termed the S-fragment, followed by a The PKs were originally predicted in 1987 and consist of two to four tandem repeats of a ~48 86 nucleotide region containing a small stem loop and downstream interaction site (figure 1B) 87 (12). Due to the sequence similarity between the PKs (figure 1C), it is speculated that they 88 were formed by duplication events during viral replication, p

In [0]:
print(format_body(file['body_text'])[:3000])



VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structural protein-coding region is replaced by reporter genes, allow the study of genome 68 replication without the requirement for high containment (9, 10) ( figure 1A ).

70

The FMDV 5′ UTR is the largest known picornavirus UTR, comprising approximately 1300 71 nucleotides and containing several highly structured regions. The first 360 nucleotides at the 5′ 72 end are predicted to fold into a single large stem loop termed the S-fragment, followed by a The PKs were originally predicted in 1987 and consist of two to four tandem repeats of a ~48 86 nucleotide region containing a small stem loop and downstream interaction site (figure 1B) 87 (12). Due to the sequence similarity between the PKs (figure 1C), it is speculated that they 88 were formed by duplication events during viral replication, p

In [0]:
print(all_files[0]['metadata'].keys())

dict_keys(['title', 'authors'])


In [0]:
print(all_files[0]['metadata']['title'])

The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus. 2 3


In [0]:
authors = all_files[0]['metadata']['authors']
pprint(authors[:3])

[{'affiliation': {},
  'email': '',
  'first': 'Joseph',
  'last': 'Ward',
  'middle': ['C'],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Lidia',
  'last': 'Lasecka-Dykes',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Chris',
  'last': 'Neil',
  'middle': [],
  'suffix': ''}]


In [0]:
for author in authors:
    print("Name:", format_name(author))
    print("Affiliation:", format_affiliation(author['affiliation']))
    print()

Name: Joseph C Ward
Affiliation: 

Name: Lidia Lasecka-Dykes
Affiliation: 

Name: Chris Neil
Affiliation: 

Name: Oluwapelumi Adeyemi
Affiliation: 

Name: Sarah 
Affiliation: 

Name:  Gold
Affiliation: 

Name: Niall Mclean
Affiliation: 

Name: Caroline Wright
Affiliation: 

Name: Morgan R Herod
Affiliation: 

Name: David Kealy
Affiliation: 

Name: Emma 
Affiliation: 

Name: Warner 
Affiliation: 

Name: Donald P King
Affiliation: 

Name: Tobias J Tuthill
Affiliation: 

Name: David J Rowlands
Affiliation: 

Name: Nicola J 
Affiliation: 

Name: Stonehouse A#
Affiliation: 



In [0]:
authors = all_files[4]['metadata']['authors']
print("Formatting without affiliation:")
print(format_authors(authors, with_affiliation=False))
print("\nFormatting with affiliation:")
print(format_authors(authors, with_affiliation=True))

Formatting without affiliation:
Nishi Kumari, Ayush Upadhyay, Kishan Kalia, Rakesh Kumar, Kanika Tuteja, Rani Paul, Eugenia Covernton, Tina Sharma, Vinod Scaria, Anshu Bhardwaj

Formatting with affiliation:
Nishi Kumari (Panjab University, Chandigarh, India), Ayush Upadhyay (Delhi University, New Delhi, India), Kishan Kalia (D.A.V. College, Sector-10, Chandigarh, India), Rakesh Kumar (CSIR-Institute of Microbial Technology, Chandigarh, India), Kanika Tuteja (Panjab University, Chandigarh, India), Rani Paul (University of Lucknow, Lucknow, India), Eugenia Covernton (Paris, France), Tina Sharma (CSIR-Institute of Microbial Technology, Chandigarh, India), Vinod Scaria, Anshu Bhardwaj (CSIR-Institute of Microbial Technology, Chandigarh, India)


In [0]:
bibs = list(file['bib_entries'].values())
pprint(bibs[:2], depth=4)

[{'authors': [{'first': 'T', 'last': 'Jackson', 'middle': [], 'suffix': ''},
              {'first': 'T', 'last': 'Tuthill', 'middle': [...], 'suffix': ''},
              {'first': 'D', 'last': 'Rowlands', 'middle': [...], 'suffix': ''},
              {'first': 'N',
               'last': 'Stonehouse',
               'middle': [...],
               'suffix': ''}],
  'issn': '',
  'other_ids': {},
  'pages': '',
  'ref_id': 'b0',
  'title': 'Genetic economy in 598 picornaviruses: Foot-and-mouth disease '
           'virus replication exploits alternative precursor 599 cleavage '
           'pathways',
  'venue': 'PLOS Pathog',
  'volume': '13',
  'year': 2017},
 {'authors': [{'first': 'N',
               'last': 'Sanderson',
               'middle': [...],
               'suffix': ''},
              {'first': 'N', 'last': 'Knowles', 'middle': [...], 'suffix': ''},
              {'first': 'D', 'last': 'King', 'middle': [...], 'suffix': ''},
              {'first': 'E', 'last': 'Cottam', 

In [0]:
format_authors(bibs[1]['authors'], with_affiliation=False)

'N D Sanderson, N J Knowles, D P King, E M Cottam'

In [0]:
cleaned_files = []

for file in tqdm(all_files):
    features = [
        file['paper_id'],
        file['metadata']['title'],
        format_authors(file['metadata']['authors']),
        format_authors(file['metadata']['authors'], 
                       with_affiliation=True),
        format_body(file['abstract']),
        format_body(file['body_text']),
        format_bib(file['bib_entries']),
        file['metadata']['authors'],
        file['bib_entries']
    ]
    
    cleaned_files.append(features)

In [0]:
col_names = [
    'paper_id', 
    'title', 
    'authors',
    'affiliations', 
    'abstract', 
    'text', 
    'bibliography',
    'raw_authors',
    'raw_bibliography'
]

clean_df = pd.DataFrame(cleaned_files, columns=col_names)
clean_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,"Joseph C Ward, Lidia Lasecka-Dykes, Chris Neil...","Joseph C Ward, Lidia Lasecka-Dykes, Chris Neil...",Abstract\n\nword count: 194 22 Text word count...,"\n\nVP3, and VP0 (which is further processed t...",Genetic economy in 598 picornaviruses: Foot-an...,"[{'first': 'Joseph', 'middle': ['C'], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Genetic..."
1,00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,"Carla Mavian, Simone Marini, Costanza Manes, I...","Carla Mavian (University of Florida, Gainesvil...","Abstract\n\nDuring the past three months, a ne...","\n\nIn December 2019, a novel coronavirus, SAR...","Situation Report -43, , Coronavirus disease 20...","[{'first': 'Carla', 'middle': [], 'last': 'Mav...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Situati..."
2,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,"Hanchu Zhou, Jiannan Yang, Kaicheng Tang, † , ...","Hanchu Zhou (City University of Hong Kong, Hon...",,Introduction\n\nThe 2019-nCoV epidemic has spr...,World Health Organizations. Novel Coronavirus ...,"[{'first': 'Hanchu', 'middle': [], 'last': 'Zh...","{'BIBREF0': {'ref_id': 'b0', 'title': 'World H..."
3,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...","Salman L Butt, Eric C Erwood, Jian Zhang, Holl...","Salman L Butt (University of Georgia, 30602, A...",Abstract\n\nInfectious bronchitis (IB) causes ...,"Introduction\n\nInfectious bronchitis (IB), wh...",Emergence of novel strains of avian infectious...,"[{'first': 'Salman', 'middle': ['L'], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Emergen..."
4,0139ea4ca580af99b602c6435368e7fdbefacb03,A Combined Evidence Approach to Prioritize Nip...,"Nishi Kumari, Ayush Upadhyay, Kishan Kalia, Ra...","Nishi Kumari (Panjab University, Chandigarh, I...",Abstract\n\nNipah Virus (NiV) came into limeli...,Introduction\n\nNipah is an infectious negativ...,"Molecular biology of Hendra and Nipah viruses,...","[{'first': 'Nishi', 'middle': [], 'last': 'Kum...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Molecul..."


In [0]:
clean_df.to_csv('biorxiv_clean.csv', index=False)

In [0]:
pmc_dir = 'C:\\Users\\angechowdhury\\OneDrive - Deloitte (O365D)\\Documents\\covid research\\custom_license\\'
pmc_files = load_files(pmc_dir)
pmc_df = generate_clean_df(pmc_files)
pmc_df.to_csv('clean_pmc.csv', index=False)
pmc_df.head()

NameError: name 'load_files' is not defined

In [0]:
comm_dir = 'C:\\Users\\angechowdhury\\OneDrive - Deloitte (O365D)\\Documents\\covid research\\comm_use_subset\\'
comm_files = load_files(comm_dir)
comm_df = generate_clean_df(comm_files)
comm_df.to_csv('clean_comm_use.csv', index=False)
comm_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,000b7d1517ceebb34e1e3e817695b6de03e2fa78,Supplementary Information An eco-epidemiologic...,"Julien Mélade, Nicolas Wieseke 4#, Beza Ramazi...","Julien Mélade (2 rue Maxime Rivière, 97490 Sai...",,\n\n- Figure S1 : Phylogeny of all sequences b...,"NDV/HQ266603/Chicken/1992, , , None; MuV/FJ375...","[{'first': 'Julien', 'middle': [], 'last': 'Mé...","{'BIBREF32': {'ref_id': 'b32', 'title': 'NDV/H..."
1,00142f93c18b07350be89e96372d240372437ed9,immunity to pathogens taught by specialized hu...,"Elisabetta Padovan, Marina Cella, Shahram Sale...","Elisabetta Padovan, Marina Cella, Shahram Sale...",Abstract\n\nDendritic cells (DCs) are speciali...,\n\niNTRODUCTiON Human beings are constantly e...,The dendritic cell system and its role in immu...,"[{'first': 'Elisabetta', 'middle': [], 'last':...","{'BIBREF0': {'ref_id': 'b0', 'title': 'The den..."
2,0022796bb2112abd2e6423ba2d57751db06049fb,Public Health Responses to and Challenges for ...,"Elvina Viennet, Scott A Ritchie, Craig R Willi...",Elvina Viennet (The Australian National Univer...,Abstract\n\nDengue has a negative impact in lo...,Introduction\n\nPathogens and vectors can now ...,"The global distribution and burden of dengue, ...","[{'first': 'Elvina', 'middle': [], 'last': 'Vi...","{'BIBREF0': {'ref_id': 'b0', 'title': 'The glo..."
3,0031e47b76374e05a18c266bd1a1140e5eacb54f,The fecal microbiota of healthy donor horses a...,"Caroline A Mckinney Id, Bruno C M Oliveira, Da...",Caroline A Mckinney Id (Cummings School of Vet...,Abstract\n\nFecal microbial transplantation (F...,\n\na1111111111 a1111111111 a1111111111 a11111...,Characteristics and risk factors for failure o...,"[{'first': 'Caroline', 'middle': ['A'], 'last'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Charact..."
4,00326efcca0852dc6e39dc6b7786267e1bc4f194,a section of the journal Frontiers in Pediatri...,"Jan Hau Lee, Oguz Dursun, Phuc Huu Phan, Yek K...","Jan Hau Lee, Oguz Dursun, Phuc Huu Phan, Yek K...","Abstract\n\nFifteen years ago, United Nations ...",\n\nIn addition to preventative care and nutri...,"Global, regional, and national levels of neona...","[{'first': 'Jan', 'middle': ['Hau'], 'last': '...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Global,..."


In [0]:
noncomm_dir = 'C:\\Users\\angechowdhury\\OneDrive - Deloitte (O365D)\\Documents\\covid research\\noncomm_use_subset\\'
noncomm_files = load_files(noncomm_dir)
noncomm_df = generate_clean_df(noncomm_files)
noncomm_df.to_csv('clean_noncomm_use.csv', index=False)
noncomm_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,0036b28fddf7e93da0970303672934ea2f9944e7,The fecal microbiota and unconjugated fecal bi...,"B Ruggerone, A C Manchester, F Del Baldo, F Fr...","B Ruggerone (University of Thessaly, Karditsa,...",Abstract\n\nand Blautia (P = 0.008) significan...,\n\nhuman type 1 DM. The aim of this study was...,Significant Feline Proteinuria: a retrospectiv...,"[{'first': 'B', 'middle': [], 'last': 'Ruggero...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Signifi..."
1,005c43980edf3fcc2a4d12ee7ad630ddb651ce6e,Development of a smartphone-based rapid dual f...,"Seon-Ju Yeo, Homan Kang, Tung Duy Dao, Thi Bui...","Seon-Ju Yeo (Wonkwang University, 570-749, Iks...",Abstract\n\nAccurate and rapid diagnosis of hi...,Introduction\n\nAvian influenza (AI) viruses a...,"Avian and other zoonotic influenza, , , 2018; ...","[{'first': 'Seon-Ju', 'middle': [], 'last': 'Y...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Avian a..."
2,00676f1131e03ca8defe523e79bc1635cc933909,Dromedary camels in northern Mali have high se...,"Darryl Falzarano, Badian Kamissoko, Emmie De W...","Darryl Falzarano (Rocky Mountain Laboratories,...",Abstract\n\nA high percentage (up to 90%) of d...,Introduction\n\nDromedary camels (Camelus drom...,Human infection with MERS coronavirus after ex...,"[{'first': 'Darryl', 'middle': [], 'last': 'Fa...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Human i..."
3,006be99e337c84b8758591a54f0362353b24dfde,Regulated Export of a Secretory Protein from t...,Stephen S Macintyre,Stephen S Macintyre (Case Western Reserve Univ...,"Abstract\n\n253 protein, C-reactive protein (C...",\n\nAbstract. The half-time for secretion of t...,Studies on the binding specificity of human C-...,"[{'first': 'Stephen', 'middle': ['S'], 'last':...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Studies..."
4,00a00d0edc750db4a0c299dd1ec0c6871f5a4f24,,,,,Editorial\n\nThis is an Open Access article di...,"Report of the Ebola Interim Assessment Panel, ...",[],"{'BIBREF0': {'ref_id': 'b0', 'title': 'Report ..."


# Create Helper Function 

In [0]:
# Helper Function for Filtering DF on Abstract + Text Substring
def abstract_text_filter(df, search_string):
    return (df.abstract.str.lower().str.replace('-', ' ')
            .str.contains(search_string, na=False) |
            df.text.str.lower().str.replace('-', ' ')
            .str.contains(search_string, na=False))

In [0]:
# Helper Function which counts Synonyms and Adds Tag Column to DF
def count_and_tag(df: pd.DataFrame,
                  synonym_list: list,
                  tag_suffix: str) -> (pd.DataFrame, pd.Series):
    counts = {}
    df[f'tag_{tag_suffix}'] = False
    for s in synonym_list:
        synonym_filter = abstract_text_filter(df, s)
        counts[s] = sum(synonym_filter)
        df.loc[synonym_filter, f'tag_{tag_suffix}'] = True
    print(f'Added tag_{tag_suffix} to DataFrame')
    return df, pd.Series(counts)

In [0]:
# Function for printing out 'n' Key Passages of Abstract based on Key Terms
def print_key_phrases(df, key_terms, n=5, chars=300):
    for ind, item in enumerate(df[:n].itertuples()):
        print(f'{ind+1} of {len(df)}')
        print(item.text)
        try:
            i = len(item.abstract)
            for kt in key_terms:
                kt = kt.replace(r'\b', '')
                term_loc = item.abstract.lower().find(kt)
                if term_loc != -1:
                    i = min(i, term_loc)
            if i < len(item.abstract):
                print('    "' + item.abstract[i-30:i+chars-30] + '"')
            else:
                print('    "' + item.abstract[:chars] + '"')
        except:
            print('NO ABSTRACT')
        print('---')

#Upload the 4 datasets 

###Dataset 1 (biorxiv)

In [0]:
# For getting CSV in Google Colab

from google.colab import files
uploaded = files.upload()


Saving biorxiv_clean.csv to biorxiv_clean (1).csv


In [0]:
import io
import pandas as pd

df1_clean = pd.read_csv(io.BytesIO(uploaded['biorxiv_clean.csv']))

In [0]:
df1_clean = pd.read_csv('biorxiv_clean.csv')

In [0]:
#View the columns in this dataset
df1_clean.columns

Index(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'],
      dtype='object')

In [0]:
#View the first 5 article in biorxiv_clean dataset

df1_clean.loc[: , "text"].head(5)

0    \n\nVP3, and VP0 (which is further processed t...
1    \n\nIn December 2019, a novel coronavirus, SAR...
2    Introduction\n\nThe 2019-nCoV epidemic has spr...
3    Introduction\n\nInfectious bronchitis (IB), wh...
4    Introduction\n\nNipah is an infectious negativ...
Name: text, dtype: object

In [0]:
# Only consider the article abstract and body 

df1 = df1_clean[['abstract', 'text']]


###Dataset 2 (Clean_pmc)


In [0]:
# For getting CSV in Google Colab

from google.colab import files
uploaded = files.upload()


Saving clean_pmc.csv to clean_pmc.csv


In [0]:
import io
import pandas as pd

df2_clean = pd.read_csv(io.BytesIO(uploaded['clean_pmc.csv']))

In [0]:
df2_clean = pd.read_csv('clean_pmc.csv')

In [0]:
#View the columns in this dataset
df2_clean.columns

Index(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'],
      dtype='object')

In [0]:
#View the first 5 article in biorxiv_clean dataset

df2_clean.loc[: , "text"].head(5)

0    Introduction\n\nAcute otitis media (AOM) is th...
1    \n\nWe identified a nurse who was asymptomatic...
2    R ift Valley fever (RVF) is a zoonotic arbovir...
3    \n\npatient group, a further significant incre...
4    M any respirators certifi ed by the National I...
Name: text, dtype: object

In [0]:
# Only consider the article abstract and body 

df2 = df1_clean[['abstract', 'text']]


###Dataset 3 (clean_noncomm_use)

In [0]:
# For getting CSV in Google Colab

from google.colab import files
uploaded = files.upload()


Saving clean_noncomm_use.csv to clean_noncomm_use.csv


In [0]:
import io
import pandas as pd

df3_clean = pd.read_csv(io.BytesIO(uploaded['clean_noncomm_use.csv']))

In [0]:
df3_clean = pd.read_csv('clean_noncomm_use.csv')

In [0]:
#View the columns in this dataset
df3_clean.columns

Index(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'],
      dtype='object')

In [0]:
#View the first 5 article in biorxiv_clean dataset

df3_clean.loc[: , "text"].head(5)

0    \n\nhuman type 1 DM. The aim of this study was...
1    Introduction\n\nAvian influenza (AI) viruses a...
2    Introduction\n\nDromedary camels (Camelus drom...
3    \n\nAbstract. The half-time for secretion of t...
4    Editorial\n\nThis is an Open Access article di...
Name: text, dtype: object

In [0]:
# Only consider the article abstract and body 

df3 = df3_clean[['abstract', 'text']]


###Dataset 4 (Clean comm use)

In [0]:
# For getting CSV in Google Colab

from google.colab import files
uploaded = files.upload()


Saving clean_comm_use.csv to clean_comm_use (2).csv


In [0]:
import io
import pandas as pd

df4_clean = pd.read_csv(io.BytesIO(uploaded['clean_comm_use.csv']))

In [0]:
df4_clean = pd.read_csv('clean_comm_use.csv')

In [0]:
#View the columns in this dataset
df4_clean.columns

Index(['paper_id', 'title', 'authors', 'affiliations', 'abstract', 'text',
       'bibliography', 'raw_authors', 'raw_bibliography'],
      dtype='object')

In [0]:
#View the first 5 article in biorxiv_clean dataset

df4_clean.loc[: , "text"].head(5)

0    \n\n- Figure S1 : Phylogeny of all sequences b...
1    \n\niNTRODUCTiON Human beings are constantly e...
2    Introduction\n\nPathogens and vectors can now ...
3    \n\na1111111111 a1111111111 a1111111111 a11111...
4    \n\nIn addition to preventative care and nutri...
Name: text, dtype: object

In [0]:
# Only consider the article abstract and body 

df4 = df4_clean[['abstract', 'text']]


#COVID-19 REGEX for Thematic Tags

##### COVID - 19
---

In limiting the search space, we target papers that specifically refer to the recent pandemic, recognised officially as COVID-19.

Source(s): https://en.wikipedia.org/wiki/Coronavirus_disease_2019

https://www.who.int/emergencies/diseases/novel-coronavirus-2019/technical-guidance/naming-the-coronavirus-disease-(covid-2019)-and-the-virus-that-causes-it

In [0]:
COVID19_SYNONYMS = ['covid',
                    'coronavirus disease 19',
                    'sars cov 2', # Note that search function replaces '-' with ' '
                    '2019 ncov',
                    '2019ncov',
                    r'2019 n cov\b',
                    r'2019n cov\b',
                    'ncov 2019',
                    r'\bn cov 2019',
                    'coronavirus 2019',
                    'wuhan pneumonia',
                    'wuhan virus',
                    'wuhan coronavirus',
                    r'coronavirus 2\b']

In [0]:
df1, covid19_counts = count_and_tag(df1, COVID19_SYNONYMS, 'disease_covid19')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Added tag_disease_covid19 to DataFrame


In [0]:
covid19_counts.sort_values(ascending=False)

covid                     492
sars cov 2                337
2019 ncov                 216
coronavirus 2\b            95
coronavirus 2019           29
ncov 2019                  10
2019ncov                   10
wuhan pneumonia             7
wuhan coronavirus           6
2019n cov\b                 4
coronavirus disease 19      4
2019 n cov\b                2
wuhan virus                 0
\bn cov 2019                0
dtype: int64

##### Ethics (Q1-4)
---
Efforts:

To articulate, translate and embed ethics to salient issues in COVID-19
To support education, accessibility and competencies across ethics
To establish a WHO team integrated with research and social science networks
In particular, we are searching for papers that specifically discuss the pervasiveness of ethics with respect to COVID-19.

Source(s): https://www.niehs.nih.gov/research/resources/bioethics/glossary/index.cfm

In [0]:
ETHICS_SYNONYMS = [
    'applied ethics',
    'assent',
    'Belmont Report', #leading work concerning Ethics and Health Care Research
    'beneficence',
    'bioethics',
    'deontology', #Ethics Theory
    'deontological ethics',
    'ethical competencies',
    'ethical conduct',
    'ethical considerations',
    'ethical dilemma',
    'ethical dimensions',
    'ethical guidance',
    'ethical reasoning',
    'ethical relativism',
    'ethical standards',
    'ethical theory',
    'ethical universalism',
    'Helsinki Declaration', #Statement of Ethical Principles guiding Clinical Research
    'Kantianism', #Ethics Theory
    'Kantian ethics',
    'Nuremberg Code', #Research Ethics Principles
    'research ethics',
    'Singapore Statement',
    'social responsibility',
    'Utilitarianism',
    'virtue ethics',
    'Working Group on Ethics' #WHO's COVID-19 Advice Team
]

df1_clean, ethics_counts = count_and_tag(df1, ETHICS_SYNONYMS, 'ethics_generic')

Added tag_ethics_generic to DataFrame


In [0]:
ethics_counts.sort_values(ascending=False)

research ethics            12
ethical considerations      4
ethical standards           3
assent                      1
social responsibility       1
ethical conduct             1
ethical dilemma             1
Working Group on Ethics     0
ethical dimensions          0
Belmont Report              0
beneficence                 0
bioethics                   0
deontology                  0
deontological ethics        0
ethical competencies        0
ethical reasoning           0
ethical guidance            0
virtue ethics               0
ethical relativism          0
ethical theory              0
ethical universalism        0
Helsinki Declaration        0
Kantianism                  0
Kantian ethics              0
Nuremberg Code              0
Singapore Statement         0
Utilitarianism              0
applied ethics              0
dtype: int64

In [0]:
n = (df.tag_disease_covid19 & df.tag_ethics_generic).sum()
print(f'There are {n} papers on COVID-19 with an "Ethical Lens".')

There are 17 papers on COVID-19 with an "Ethical Lens".


In [0]:
# Printed below are up to five examples, and their respective key texts from the Abstract.
print_key_phrases(df[df.tag_disease_covid19 & df.tag_ethics_generic],
                  ETHICS_SYNONYMS)

####Frameworks for Prevention and Control (Q5a)
---

We are searching for papers that specifically discuss information attainment with respect to the adherence to public health measures during COVID-19.




In [0]:
FRAMEWORK_SYNONYMS = [
    'qualitative assessment framework',
    'systematic collection',
    'systematic information collection'
]
df, framework_counts = count_and_tag(df, FRAMEWORK_SYNONYMS, 'ss_framework')

Added tag_ss_framework to DataFrame


In [0]:
framework_counts.sort_values(ascending=False)

systematic collection                25
systematic information collection     0
qualitative assessment framework      0
dtype: int64

In [0]:
n = (df.tag_disease_covid19 & df.tag_ss_framework).sum()
print(f'There are {n} papers detailing the development of qualitative assessment frameworks in capturing responses to COVID-19 public health measures.')

There are 0 papers detailing the development of qualitative assessment frameworks in capturing responses to COVID-19 public health measures.


In [0]:
print_key_phrases(df[df.tag_disease_covid19 & df.tag_ss_framework],
                  FRAMEWORK_SYNONYMS)

Secondary Impacts (Q5b)

---

We are searching for papers that specifically discuss the flow-on effects of public health measures during COVID-19. This may include the widespread use of surgical masks, modification of health seeking behaviours regarding sexual and reproductive health (SRH), as well as school closures.

In [0]:
IMPACTS_SYNONYMS = [
    'behavior changes',
    'behaviour changes',
    'care seeking behavior',
    'care seeking behaviour',
    'flattening the curve',
    'health seeking',
    'public compliance',
    'sexual and reproductive health',
    'school closures',
    'secondary impact',
    'secondary measures',
    'social isolation',
    'surgical masks',
    'personal protective equipment',
]
df, impacts_counts = count_and_tag(df, IMPACTS_SYNONYMS, 'ss_impacts')

Added tag_ss_impacts to DataFrame


In [0]:
impacts_counts.sort_values(ascending=False)

personal protective equipment     379
surgical masks                     90
school closures                    79
health seeking                     72
care seeking behavior              31
social isolation                   26
behavior changes                   26
care seeking behaviour             16
behaviour changes                  15
public compliance                   8
secondary measures                  5
sexual and reproductive health      4
flattening the curve                4
secondary impact                    2
dtype: int64

In [0]:
n = (df.tag_disease_covid19 & df.tag_ss_impacts).sum()
print(f'There are {n} papers detailing the secondary impacts of COVID-19 public health measures.')

There are 67 papers detailing the secondary impacts of COVID-19 public health measures.


In [0]:
# Printed below are up to five examples, and their respective key texts from the Abstract.
print_key_phrases(df[df.tag_disease_covid19 & df.tag_ss_impacts],
                  IMPACTS_SYNONYMS)

#####Burden of Care (Q6)

---
We are searching for papers that highlight the immediate needs as well as the physical and psychological health of care providers in light of COVID-19.

In [0]:
HEALTH_SYNONYMS = [
    'emotional support',
    'health care workers',
    'immediate needs',
    'medical staff',
    'mental health',
    'personal safety',
    'physical and mental health',
    'physical health',
    'psychological state',
    'psychological health',
    'psychological support',
    'wellbeing'
]
df, health_counts = count_and_tag(df, HEALTH_SYNONYMS, 'ss_health')

Added tag_ss_health to DataFrame


In [0]:
health_counts.sort_values(ascending=False)

health care workers           423
mental health                 355
medical staff                 274
wellbeing                     110
physical health                40
physical and mental health     27
psychological support          25
personal safety                21
emotional support              18
psychological health           17
psychological state             9
immediate needs                 5
dtype: int64

In [0]:
n = (df.tag_disease_covid19 & df.tag_ss_health).sum()
print(f'There are {n} papers detailing the burden of care that COVID-19 has imposed on care providers.')

There are 130 papers detailing the burden of care that COVID-19 has imposed on care providers.


In [0]:
print_key_phrases(df[df.tag_disease_covid19 & df.tag_ss_health],
                  HEALTH_SYNONYMS)

#####Social Media Fear Mongering

---
We are searching for papers that identify and evaluate the drivers of misinformation and negative emotions; specifically fear, anxiety and stigma about COVID-19, arising from social media outlets.

In [0]:
SOCIAL_SYNONYMS = [
    'anxiety',
    'anxious',
    'fear',
    'panic',
    'psychological issues',
    'stigma',
    'stigmatization',
    'overwhelming',
    'fake news',
    'mistrust',
    'misperceptions',
    'uncertainty',
    'exacerbate',
    'misunderstanding',
    'misinformation',
    'hyperbole',
    'social media',
    'worry',
    'distress',
    'emotional distress',
    'facebook posts',
    'tweets',
    'xenophobia',
    'hysteria',
    'rumor',
    'rumour'
]
df, social_counts = count_and_tag(df, SOCIAL_SYNONYMS, 'ss_social')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Added tag_ss_social to DataFrame


In [0]:
social_counts.sort_values(ascending=False)

distress                1104
uncertainty              925
exacerbate               674
fear                     603
panic                    344
anxiety                  342
overwhelming             340
social media             207
stigma                   179
worry                    171
misunderstanding          62
anxious                   60
rumor                     57
stigmatization            54
misinformation            48
rumour                    43
mistrust                  30
tweets                    25
emotional distress        20
misperceptions            13
hysteria                  10
fake news                  7
psychological issues       5
facebook posts             4
xenophobia                 2
hyperbole                  2
dtype: int64

In [0]:
n = (df.tag_disease_covid19 & df.tag_ss_social).sum()
print(f'There are {n} papers highlighting the social media drivers of COVID-19 misinformation and stigma.')

There are 390 papers highlighting the social media drivers of COVID-19 misinformation and stigma.


In [0]:
# Printed below are up to five examples, and their respective key texts from the Abstract.
print_key_phrases(df[df.tag_disease_covid19 & df.tag_ss_social],
                  SOCIAL_SYNONYMS)

# Build word embeddings from corpus 
https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c

In [0]:
# remove parenthesis

import re

df1_clean = df1.to_string()

df1_clean_noparens = re.sub(r'\([^)]*\)', '', df1_clean)


In [0]:
#check length of the string after removing parenthesis

len(df1_clean_noparens)

In [0]:
# store as list of sentences

sentences_strings_ted = []
for line in df1_clean_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

In [0]:
# store as list of lists of words

sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [0]:
# sentences_ted has been transformed into a two dimensional array with each element being a word.

# Let’s print out the first to tenth element.

sentences_ted[0:10]

In [0]:

# Install a conda package in the current Jupyter kernel

import sys
!pip install --yes --prefix {sys.prefix} gensim


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --yes


In [0]:
# This is the form that is ready to be fed into the Word2Vec model defined in Gensim. 
# Word2Vec model can be easily trained with one line as the code below.

from gensim.models import Word2Vec
model_ted_w2v = Word2Vec(sentences=sentences_ted, size=100, window=20, min_count=5, workers=4, sg=0)

#sentences: the list of split sentences.
#size: the dimensionality of the embedding vector
#window: the number of context words you are looking at
#min_count: tells the model to ignore words with total count less than this number.
#workers: the number of threads being used
#sg: whether to use skip-gram or CBOW

In [0]:
#Now, let’s try which words are most similar to the word found to be the most prominant COVID-19 REGEX for Thematic Tags section

result_w2v = model_ted_w2v.wv.most_similar(['covid', 'virus', 'wuhan', 
                                    'pneumonia', 'coronavirus', 
                                    'ethics', 'social', 'school', 
                                    'personal', 'protective', 
                                    'equipment', 'health', 'behavior',
                                    'flattening','curve','surgical','masks',
                                    'isolation', 'secondary', 'impact', 
                                    'medical', 'health', 'mental', 'wellbeing',
                                    'psychological', 'emotional', 'physical',
                                    'uncertainty', 'distress', 'exacerbate','fear',
                                    'panic', 'anxiety', 'media', 'overwhelming', 
                                    'worry'],topn=200)
for a, b in result_w2v:  
    print(a,":", b)

In [0]:
#Fasttext

from gensim.models import FastText
model_ted_ft = FastText(sentences_ted, size=100, window=20, min_count=5, workers=4,sg=1)

In [0]:
#Now, let’s try which words are most similar to the word "covid".

result_ft = model_ted_ft.wv.most_similar(['covid', 'virus', 'wuhan', 
                                    'pneumonia', 'coronavirus', 
                                    'ethics', 'social', 'school', 
                                    'personal', 'protective', 
                                    'equipment', 'health', 'behavior',
                                    'flattening','curve','surgical','masks',
                                    'isolation', 'secondary', 'impact', 
                                    'medical', 'health', 'mental', 'wellbeing',
                                    'psychological', 'emotional', 'physical',
                                    'uncertainty', 'distress', 'exacerbate','fear',
                                    'panic', 'anxiety', 'media', 'overwhelming', 
                                    'worry'],topn=200)
for a, b in result_ft:  
    print(a,":", b)

P.S: We need to try out the models with the following keyword (from Cindy's Research)

“information”
“confidential”
“conflict”
“informed”
“coerce”
“review”
“benefit”
“risk”
“randomise”
“confidence”
“significance”
“intervention”
“blind”
“researchers”
“life”
“cost” 
“trial”
“therapeutic”
“regulation”
“FDA”
“EMA”
“expedited”
“evidence”
“size”
“difference”
“power”
